# Little Washita Worked Example
## ParFlow Shortcourse June 2022
This notebook translates the tcl for exercise four of the ParFlow shortcourse to the newest parflow-python integration. 
<br /> The end result is 72 hourly outputs from the X forcing year for the Little Washita domain. 

Import required libraries and functions to run ParFlow in Python. 
<br />Documentation for pftools can be found at https://github.com/parflow/parflow/tree/master/pftools/python

In [35]:
import sys
import os
import numpy as np
from parflow import Run 
import shutil
from parflow.tools.fs import mkdir, get_absolute_path
from parflow.tools.fs import mkdir, cp, get_absolute_path, exists
from parflow.tools.settings import set_working_directory

Initialize paths in order to read input and write output files. 
<br />Additionally, set the runname and initialize the model object whose attribute we will set in following cells. 

In [36]:
PROJECT_PATH = os.getcwd() #there is some un-optimability with how the wd is beigng set, when i move around this can change sometimes and cause unexpected behavior
os.chdir(PROJECT_PATH)
PFINPUT_DIRECTORY = f"{PROJECT_PATH}/LW/parflow_input"
CLMINPUT_DIRECTORY = f"{PROJECT_PATH}/LW/clm_input"
DOMAIN_DIRECTORY = f"{PROJECT_PATH}/LW/outputs"
model = Run("LW_CLM_Ex4", DOMAIN_DIRECTORY )
model.FileVersion = 4

Before we really get started, copy all of the required input files into the run directory. 
<br />These files will be described in detail later as they get used.

In [37]:
#Copy Parflow inputs
files=os.listdir(PFINPUT_DIRECTORY)
# iterating over all the files parflow_inputs dir
for fname in files:
    # copying the files to the destination directory
    shutil.copy(os.path.join(PFINPUT_DIRECTORY,fname), DOMAIN_DIRECTORY)
    
#Copy clm inputs
files=os.listdir(CLMINPUT_DIRECTORY)
# iterating over all the files parflow_inputs dir
for fname in files:
    # copying the files to the destination directory
    shutil.copy(os.path.join(CLMINPUT_DIRECTORY,fname), DOMAIN_DIRECTORY)  

#### Parallel Process Topology. 
The domain is divided in x,y and z by P, Q and R. The total number of processors is P*Q*R.

In [38]:
model.Process.Topology.P = 1
model.Process.Topology.Q = 1
model.Process.Topology.R = 1

#### Computational Grid:

In [39]:
#Locate the origin in the domain.
model.ComputationalGrid.Lower.X = 0.0
model.ComputationalGrid.Lower.Y = 0.0
model.ComputationalGrid.Lower.Z = 0.0
# Define the size of the domain grid block. Length units, same as those on hydraulic conductivity, here that is meters.
model.ComputationalGrid.DX = 1000.0
model.ComputationalGrid.DY = 1000.0
model.ComputationalGrid.DZ = 200.0
# Define the number of grid blocks in the domain.
model.ComputationalGrid.NX = 64
model.ComputationalGrid.NY = 32
model.ComputationalGrid.NZ = 10

#### The Names of the GeomInputs
This next piece is comparable to a pre-declaration of variables. These will be areas in our domain geometry. 
<br />The regions themselves will be defined later. You must always have one that is the name of your entire domain. 
<br />If you want subsections within your domain, you may declare these as well. 
<br />Here we define two geometries one is the domain, here solid_input because we will use a solid file to represent the domain 
<br />and one is for the indicator file (which will also span the entire domain).

In [40]:
model.GeomInput.Names = "solid_input indi_input"

#### Domain Geometry Input
Now you characterize the domain that you just pre-declared to be solid file input, with file extension .pfsol. 

In [41]:
model.GeomInput.Names = "solid_input indi_input"
model.GeomInput.solid_input.InputType = "SolidFile"
model.GeomInput.solid_input.GeomNames = "domain"
model.GeomInput.solid_input.FileName = "LW.pfsol"


#### Domain Geometry
Here, you would usually set the limits in space for your entire domain, but it is not necessary to explicitly do this as we are using a solid file. 
<br />The Patches key assigns names to the outside edges, because the domain is the limit of the problem in space. Each patch refers to a different value region in the solid file.

In [42]:
model.Geom.domain.Patches = "top bottom side"

#### Variable Dz
Variable Dz allows you to change the thickness of each of your z-layers, the default is a uniform thickness. 
<br />In this example, the DZ is 200 and the NZ is 10, meaning the default is 10 layers, which are 200m in thickness each. 
<br />The .dzscale.Value lines are numbers by which is uniform 200m will be scaled for each layer. 
<br />For ParFlow, 0 refers to the bottom layer which means that the bottom layer will have a thickness of 1000m (200 * 5). 
<br />The top layer (9) with a scalar of .0005 will have a thickness of 0.1m (.0005 * 200m)

In [43]:
model.Solver.Nonlinear.VariableDz = True
model.dzScale.GeomNames = "domain"
model.dzScale.Type = "nzList"
model.dzScale.nzListNumber = 10

model.Cell._0.dzScale.Value = 5
model.Cell._1.dzScale.Value = 0.5
model.Cell._2.dzScale.Value = 0.25
model.Cell._3.dzScale.Value = 0.125
model.Cell._4.dzScale.Value = 0.05
model.Cell._5.dzScale.Value = 0.025
model.Cell._6.dzScale.Value = 0.005
model.Cell._7.dzScale.Value = 0.003
model.Cell._8.dzScale.Value = 0.0015
model.Cell._9.dzScale.Value = 0.0005

#### Indicator Geometry Input
Now we setup the indicator file. As noted above, the indicator file has integer values for every grid cell in the domain designating what geologic unit it belongs to. 
<br />The GeomNames list should include a name for every unit in your indicator file. 
<br />In this example we have thirteen soil units (s1-s13) and ten geologic units (g1-g13). 
<br />The FileName points to the indicator file that ParFlow will read. 
<br />Recall that this file is copied into the run directory at the start of the script.

In [44]:
model.GeomInput.indi_input.InputType =   "IndicatorField"
model.GeomInput.indi_input.GeomNames = "s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13 g1 g2 g3 g4 g5 g6 g7 g8 g9 g10"
model.Geom.indi_input.FileName = "Indicator_LW_USGS_Bedrock.pfb"

For every name in the GeomNames list we define the corresponding value in the indicator file. 
<br />For example, here we are saying that our first soil unit (s1) is represented by the number “1" in the indicator file, while the first geologic unit (g1) is represented by the number “19". <br />Note: that the integers used in the indicator file do not need to be consecutive. 

In [45]:
model.GeomInput.s1.Value =    1
model.GeomInput.s2.Value =    2
model.GeomInput.s3.Value =    3
model.GeomInput.s4.Value =    4
model.GeomInput.s5.Value =    5
model.GeomInput.s6.Value =    6
model.GeomInput.s7.Value =    7
model.GeomInput.s8.Value =    8
model.GeomInput.s9.Value =    9
model.GeomInput.s10.Value =   10
model.GeomInput.s11.Value =   11
model.GeomInput.s12.Value =   12
model.GeomInput.s13.Value =   13

model.GeomInput.g1.Value =    19
model.GeomInput.g2.Value =    20
model.GeomInput.g3.Value =    21
model.GeomInput.g4.Value =    22
model.GeomInput.g5.Value =    23
model.GeomInput.g6.Value =    24
model.GeomInput.g7.Value =    25
model.GeomInput.g8.Value =    26
model.GeomInput.g9.Value =    27
model.GeomInput.g10.Value =    28


#### Permeability (values in m/hr)
Now you add permeability (k) data to the domain sections defined above. 
<br />You can reassign values simply by re-stating them – there is no need to comment out or delete the previous version – the final statement is the only one that counts. 
<br />Also, note that you do not need to assign permeability values to all of the geometries names. 
<br />Any geometry that is not assigned its own permeability value will take the domain value. 
<br />However, every geometry listed in Porosity.GeomNames must have values assigned.

In [46]:
model.Geom.Perm.Names = "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13 g1 g2 g3 g4 g5 g6 g7 g8 g9 g10"

model.Geom.domain.Perm.Type = "Constant"
model.Geom.domain.Perm.Value = 0.02

model.Geom.s1.Perm.Type = "Constant"
model.Geom.s1.Perm.Value = 0.269022595

model.Geom.s2.Perm.Type = "Constant"
model.Geom.s2.Perm.Value = 0.043630356

model.Geom.s3.Perm.Type = "Constant"
model.Geom.s3.Perm.Value = 0.015841225

model.Geom.s4.Perm.Type = "Constant"
model.Geom.s4.Perm.Value = 0.007582087

model.Geom.s5.Perm.Type = "Constant"
model.Geom.s5.Perm.Value = 0.01818816

model.Geom.s6.Perm.Type = "Constant"
model.Geom.s6.Perm.Value = 0.005009435

model.Geom.s7.Perm.Type = "Constant"
model.Geom.s7.Perm.Value = 0.005492736

model.Geom.s8.Perm.Type = "Constant"
model.Geom.s8.Perm.Value = 0.004675077

model.Geom.s9.Perm.Type = "Constant"
model.Geom.s9.Perm.Value = 0.003386794

model.Geom.s10.Perm.Type = "Constant"
model.Geom.s10.Perm.Value = 0.004783973

model.Geom.s11.Perm.Type = "Constant"
model.Geom.s11.Perm.Value = 0.003979136

model.Geom.s12.Perm.Type = "Constant"
model.Geom.s12.Perm.Value = 0.006162952

model.Geom.s13.Perm.Type = "Constant"
model.Geom.s13.Perm.Value = 0.005009435

model.Geom.g1.Perm.Type = "Constant"
model.Geom.g1.Perm.Value = 5e-3

model.Geom.g2.Perm.Type = "Constant"
model.Geom.g2.Perm.Value = 1e-2

model.Geom.g3.Perm.Type = "Constant"
model.Geom.g3.Perm.Value = 2e-2

model.Geom.g4.Perm.Type = "Constant"
model.Geom.g4.Perm.Value = 3e-2

model.Geom.g5.Perm.Type = "Constant"
model.Geom.g5.Perm.Value = 4e-2

model.Geom.g6.Perm.Type = "Constant"
model.Geom.g6.Perm.Value = 5e-2

model.Geom.g7.Perm.Type = "Constant"
model.Geom.g7.Perm.Value = 6e-2

model.Geom.g8.Perm.Type = "Constant"
model.Geom.g8.Perm.Value = 8e-2

model.Geom.g9.Perm.Type = "Constant"
model.Geom.g9.Perm.Value = 0.1

model.Geom.g10.Perm.Type = "Constant"
model.Geom.g10.Perm.Value = 0.2

The following section allows you to specify the permeability tensor. In the case below, permeability is symmetric in all directions (x, y, and z) and therefore each is set to 1.0. 
<br />Also note that we just specify this once for the whole domain because we want isotropic permeability everywhere. 
<br />You can specify different tensors for different units by repeating these lines with different Geom.Names.

In [47]:
model.Perm.TensorType = "TensorByGeom"
model.Geom.Perm.TensorByGeom.Names = "domain"
model.Geom.domain.Perm.TensorValX = 1.0
model.Geom.domain.Perm.TensorValY = 1.0
model.Geom.domain.Perm.TensorValZ = 1.0

#### Specific Storage
Next we set the specific storage. Here again we specify one value for the whole domain but these lines can be easily repeated to set different values for different units.

In [48]:
model.SpecificStorage.Type = "Constant"
model.SpecificStorage.GeomNames = "domain"
model.Geom.domain.SpecificStorage.Value = 0.0001

#### Phases
ParFlow has the capability to deal with a multiphase system, but we only have one (water) in this example. 
<br />As we stated earlier, we set density and viscosity artificially (and later gravity) both to 1.0. 
<br />Again, this is merely a trick to solve for hydraulic conductivity and pressure head. 
<br />If you were to set density and viscosity to their true values, the code would calculate k (permeability). 
<br />By using the normalized values instead, you effectively embed the conversion of k to K (hydraulic conductivity). 
<br />So this way, we get hydraulic conductivity, which is what we want for this problem.

In [49]:
model.Phase.Names = "water"
model.Phase.water.Density.Type = "Constant"
model.Phase.water.Density.Value = 1.0
model.Phase.water.Viscosity.Type = "Constant"
model.Phase.water.Viscosity.Value = 1.0

#### Contaminants
This example does not include the ParFlow grid based transport scheme. Therefore we leave contaminants blank.

In [50]:
model.Contaminants.Names = ""

#### Gravity
As with density and viscosity, gravity is normalized here. If we used the true value (in the [L] and [T] units of hydraulic conductivity) the code would be calculating permeability (k). 
<br />Instead, we normalize so that the code calculates hydraulic conductivity (K).

In [51]:
model.Gravity = 1.0

#### Timing (time units is set by units of permeability)
This specifies the base unit of time for all time values entered. All time should be expressed as multiples of this value. 
<br />To keep things simple here we set it to 1. Because we expressed our permeability in units of m/hr in this example this means that our base unit of time is 1hr.

In [52]:
model.TimingInfo.BaseUnit = 1.0

This key specifies the time step number that will be associated with the first advection cycle of the transient problem. 
<br />Because we are starting from scratch we set this to 0. If we were restarting a run we would set this to the last time step of your previous simulation. 
<br />Refer to § 3.3 for additional instructions on restarting a run.

In [53]:
model.TimingInfo.StartCount = 0

StartTime and StopTime specify the start and stop times for the simulation. These values should correspond with the forcing files you are using.
Here the stop time is set to 72 hours or three days. 

In [54]:
model.TimingInfo.StartTime = 0
model.TimingInfo.StopTime = 72.0

This key specifies the timing interval at which ParFlow time dependent outputs will be written. 
<br />Here we have a base unit of 1hr so a dump interval of 24 means that we are writing daily outputs. 
<br />Note that this key only controls the ParFlow output interval and not the interval that CLM outputs will be written out at.

In [55]:
model.TimingInfo.DumpInterval = 1.0

Here we set the time step value. For this example we use a constant time step of 1hr.

In [56]:
model.TimeStep.Type = "Constant"
model.TimeStep.Value = 1.0

#### Porosity
Next, we assign the porosity (see § 6.1.12). As with the permeability we assign different values for different indicator geometries. 
<br />Here we assign values for all of our soil units but not for the geologic units, they will default to the domain value of 0.33. 
<br />Note that every geometry listed in Porosity.GeomNames must have values assigned which is why we do not list the geologic named units.

In [57]:
model.Geom.Porosity.GeomNames = "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13"

model.Geom.domain.Porosity.Type = "Constant"
model.Geom.domain.Porosity.Value = 0.33

model.Geom.s1.Porosity.Type = "Constant"
model.Geom.s1.Porosity.Value = 0.375

model.Geom.s2.Porosity.Type = "Constant"
model.Geom.s2.Porosity.Value = 0.39

model.Geom.s3.Porosity.Type = "Constant"
model.Geom.s3.Porosity.Value = 0.387

model.Geom.s4.Porosity.Type = "Constant"
model.Geom.s4.Porosity.Value = 0.439

model.Geom.s5.Porosity.Type = "Constant"
model.Geom.s5.Porosity.Value = 0.489

model.Geom.s6.Porosity.Type = "Constant"
model.Geom.s6.Porosity.Value = 0.399

model.Geom.s7.Porosity.Type = "Constant"
model.Geom.s7.Porosity.Value = 0.384

model.Geom.s8.Porosity.Type = "Constant"
model.Geom.s8.Porosity.Value = 0.482

model.Geom.s9.Porosity.Type = "Constant"
model.Geom.s9.Porosity.Value = 0.442

model.Geom.s10.Porosity.Type = "Constant"
model.Geom.s10.Porosity.Value = 0.385

model.Geom.s11.Porosity.Type = "Constant"
model.Geom.s11.Porosity.Value = 0.481

model.Geom.s12.Porosity.Type = "Constant"
model.Geom.s12.Porosity.Value = 0.459

model.Geom.s13.Porosity.Type = "Constant"
model.Geom.s13.Porosity.Value = 0.399

# model.Geom.g1.Porosity.Type =           "Constant"
# model.Geom.g1.Porosity.Value =           0.33

# model.Geom.g2.Porosity.Type =           "Constant"
# model.Geom.g2.Porosity.Value =           0.33

# model.Geom.g3.Porosity.Type =           "Constant"
# model.Geom.g3.Porosity.Value =           0.30

# model.Geom.g4.Porosity.Type =           "Constant"
# model.Geom.g4.Porosity.Value =           0.30

# model.Geom.g5.Porosity.Type =           "Constant"
# model.Geom.g5.Porosity.Value =           0.10

# model.Geom.g6.Porosity.Type =           "Constant"
# model.Geom.g6.Porosity.Value =           0.30

# model.Geom.g7.Porosity.Type =           "Constant"
# model.Geom.g7.Porosity.Value =           0.30

# model.Geom.g8.Porosity.Type =           "Constant"
# model.Geom.g8.Porosity.Value =           0.30

#### Domain
Having defined the geometry of our problem before and named it domain, we are now ready to report/upload that problem, which we do here.

In [58]:
model.Domain.GeomName = "domain"

#### Mobility
Mobility between phases is set to 1.0 because we only have one phase (water).

In [59]:
model.Phase.water.Mobility.Type = "Constant"
model.Phase.water.Mobility.Value = 1.0

#### Wells
Again, ParFlow has more capabilities than we are using here in this example. Note that since there are no well names listed here, ParFlow assumes we have no wells. 
<br />If we had pumping wells, we would have to include them here, because they would affect the head distribution throughout our domain. 
<br />See § 3.6.1 for an example of how to include pumping wells in this script.

In [60]:
model.Wells.Names = ""

#### Time Cycles
You can give certain periods of time names if you want. For example if you aren’t running with CLM and you would like to have periods with rain and periods without. 
<br />Here, however we have only one time cycle because CLM will handle the variable forcings. 
<br />Therefore, we specify one time cycle and it’s constant for the duration of the simulation.
<br />We accomplish this by giving it a repeat value of -1, which repeats indefinitely. 
<br />The length of the cycle is the length specified below (an integer) multiplied by the base unit value we specified earlier.

In [61]:
# model.Cycle.Names ="constant rainrec"
model.Cycle.Names ="constant"
model.Cycle.constant.Names = "alltime"
model.Cycle.constant.alltime.Length = 1
model.Cycle.constant.Repeat = -1

# rainfall and recession time periods are defined here
# rain for 1 hour, recession for 2 hours
# model.Cycle.rainrec.Names = "rain rec"
# model.Cycle.rainrec.rain.Length = 1
# model.Cycle.rainrec.rec.Length = 5000000
# model.Cycle.rainrec.Repeat = -1

#### Boundary Conditions
Now, we assign Boundary Conditions for each face (each of the Patches in the domain defined before). 
<br />Recall the previously stated Patches and associate them with the boundary conditions that follow.
<br />The bottom and sides of our domain are all set to no-flow (i.e. constant flux of 0) boundaries.
<br />The top is set to an OverlandFLow boundary to turn on the fully-coupled overland flow routing, in this case, overland kinematic wave approximation.

In [62]:
model.BCPressure.PatchNames = "top bottom side"

model.Patch.bottom.BCPressure.Type = "FluxConst"
model.Patch.bottom.BCPressure.Cycle = "constant"
model.Patch.bottom.BCPressure.alltime.Value = 0.0

model.Patch.side.BCPressure.Type = "FluxConst"
model.Patch.side.BCPressure.Cycle = "constant"
model.Patch.side.BCPressure.alltime.Value = 0.0

model.Patch.top.BCPressure.Type = "OverlandKinematic"
model.Patch.top.BCPressure.Cycle = "constant"
model.Patch.top.BCPressure.alltime.Value = 0.0

#### Topo slopes in x- and y- direction
Next we define topographic slopes and values. These slope values were derived from a digital elevation model of the domain following the workflow in the PriorityFlow github package.
<br />In this example we read the slope files in from .pfb files that were copied into the run directory at the start of this script.

In [63]:
model.TopoSlopesX.Type = "PFBFile"
model.TopoSlopesX.GeomNames = "domain"
model.TopoSlopesX.FileName = "slopex_LW.pfb"

model.TopoSlopesY.Type = "PFBFile"
model.TopoSlopesY.GeomNames = "domain"
model.TopoSlopesY.FileName = "slopey_LW.pfb"

#### Mannings coefficient
And now we define the Mannings n, just one value for the whole domain in this example.

In [64]:
model.Mannings.Type = "Constant"
model.Mannings.GeomNames = "domain"
model.Mannings.Geom.domain.Value = 0.0000044

#### Relative Permeability
Following the same approach as we did for Porosity we define the relative permeability inputs that will be used for Richards’ equation implementation (§ 6.1.20). 
<br />Here we use VanGenuchten parameters. Note that every geometry listed in Porosity.GeomNames must have values assigned.
<br />Unnamed units will get the background domain value of 1.0 and 3.0

In [65]:
model.Phase.RelPerm.Type =              "VanGenuchten"
model.Phase.RelPerm.GeomNames =     "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13"

model.Geom.domain.RelPerm.Alpha =    1.0
model.Geom.domain.RelPerm.N =        3.0

model.Geom.s1.RelPerm.Alpha =        3.548
model.Geom.s1.RelPerm.N =            4.162

model.Geom.s2.RelPerm.Alpha =        3.467
model.Geom.s2.RelPerm.N =            2.738

model.Geom.s3.RelPerm.Alpha =        2.692
model.Geom.s3.RelPerm.N =            2.445

model.Geom.s4.RelPerm.Alpha =        0.501
model.Geom.s4.RelPerm.N =            2.659

model.Geom.s5.RelPerm.Alpha =        0.661
model.Geom.s5.RelPerm.N =            2.659

model.Geom.s6.RelPerm.Alpha =        1.122
model.Geom.s6.RelPerm.N =            2.479

model.Geom.s7.RelPerm.Alpha =        2.089
model.Geom.s7.RelPerm.N =            2.318

model.Geom.s8.RelPerm.Alpha =        0.832
model.Geom.s8.RelPerm.N =            2.514

model.Geom.s9.RelPerm.Alpha =        1.585
model.Geom.s9.RelPerm.N =            2.413

model.Geom.s10.RelPerm.Alpha =        3.311
model.Geom.s10.RelPerm.N =            2.202

model.Geom.s11.RelPerm.Alpha =        1.622
model.Geom.s11.RelPerm.N =            2.318

model.Geom.s12.RelPerm.Alpha =        1.514
model.Geom.s12.RelPerm.N =            2.259

model.Geom.s13.RelPerm.Alpha =        1.122
model.Geom.s13.RelPerm.N =            2.479

#### Saturation
Next we do the same thing for saturation (§ 6.1.23) again using the VanGenuchten parameters 
<br />Note that every geometry listed in Porosity.GeomNames must have values assigned.
<br />Unnamed units will get the background domain values assigned

In [66]:
model.Phase.Saturation.Type =             "VanGenuchten"
model.Phase.Saturation.GeomNames =         "domain s1 s2 s3 s4 s5 s6 s7 s8 s9 s10 s11 s12 s13"

model.Geom.domain.Saturation.Alpha =        1.0
model.Geom.domain.Saturation.N =            3.0
model.Geom.domain.Saturation.SRes =         0.001
model.Geom.domain.Saturation.SSat =         1.0

model.Geom.s1.Saturation.Alpha =        3.548
model.Geom.s1.Saturation.N =            4.162
model.Geom.s1.Saturation.SRes =         0.0001
model.Geom.s1.Saturation.SSat =         1.0

model.Geom.s2.Saturation.Alpha =        3.467
model.Geom.s2.Saturation.N =            2.738
model.Geom.s2.Saturation.SRes =         0.0001
model.Geom.s2.Saturation.SSat =         1.0

model.Geom.s3.Saturation.Alpha =        2.692
model.Geom.s3.Saturation.N =            2.445
model.Geom.s3.Saturation.SRes =         0.0001
model.Geom.s3.Saturation.SSat =         1.0

model.Geom.s4.Saturation.Alpha =        0.501
model.Geom.s4.Saturation.N =            2.659
model.Geom.s4.Saturation.SRes =         0.1
model.Geom.s4.Saturation.SSat =         1.0

model.Geom.s5.Saturation.Alpha =        0.661
model.Geom.s5.Saturation.N =            2.659
model.Geom.s5.Saturation.SRes =         0.0001
model.Geom.s5.Saturation.SSat =         1.0

model.Geom.s6.Saturation.Alpha =        1.122
model.Geom.s6.Saturation.N =            2.479
model.Geom.s6.Saturation.SRes =         0.0001
model.Geom.s6.Saturation.SSat =         1.0

model.Geom.s7.Saturation.Alpha =        2.089
model.Geom.s7.Saturation.N =            2.318
model.Geom.s7.Saturation.SRes =         0.0001
model.Geom.s7.Saturation.SSat =         1.0

model.Geom.s8.Saturation.Alpha =        0.832
model.Geom.s8.Saturation.N =            2.514
model.Geom.s8.Saturation.SRes =         0.0001
model.Geom.s8.Saturation.SSat =         1.0

model.Geom.s9.Saturation.Alpha =        1.585
model.Geom.s9.Saturation.N =            2.413
model.Geom.s9.Saturation.SRes =         0.0001
model.Geom.s9.Saturation.SSat =         1.0

model.Geom.s10.Saturation.Alpha =        3.311
model.Geom.s10.Saturation.N =            2.202
model.Geom.s10.Saturation.SRes =         0.0001
model.Geom.s10.Saturation.SSat =         1.0

model.Geom.s11.Saturation.Alpha =        1.622
model.Geom.s11.Saturation.N =            2.318
model.Geom.s11.Saturation.SRes =         0.0001
model.Geom.s11.Saturation.SSat =         1.0

model.Geom.s12.Saturation.Alpha =        1.514
model.Geom.s12.Saturation.N =            2.259
model.Geom.s12.Saturation.SRes =         0.0001
model.Geom.s12.Saturation.SSat =         1.0

model.Geom.s13.Saturation.Alpha =        1.122
model.Geom.s13.Saturation.N =            2.479
model.Geom.s13.Saturation.SRes =         0.0001
model.Geom.s13.Saturation.SSat =         1.0

#### Phase sources
Phase sources allows you to add sources other than wells and boundaries, but we do not have any so this key is constant, 0.0 over entire domain.

In [67]:
model.PhaseSources.water.Type = "Constant"
model.PhaseSources.water.GeomNames = "domain"
model.PhaseSources.water.Geom.domain.Value = 0.0

#### CLM Settings
In this example we are using ParFlow CLM so we must provide some parameters for CLM (§ 6.1.36). 
<br />Note that CLM will also require some additional inputs outside of the tcl script. Refer to /washita/clm_input/ for examples of the CLM vegm and driver files. 
<br />These inputs are also discussed briefly in § 3.1.2.
<br />
<br />
First we specify that we will be using CLM as the land surface model and provide the name of a directory that outputs will be written to. 
<br />For this example we do not need outputs for each processor or a binary output directory. 
<br />Finally we set the dump interval to 1, indicating that we will be writing outputs for every time step. 
<br />Note that this does not have to match the dump interval for ParFlow outputs. 

In [68]:
model.Solver.LSM = "CLM"
model.Solver.CLM.CLMFileDir = "clm_output/"
model.Solver.CLM.Print1dOut = False
model.Solver.BinaryOutDir = False #Solver: Field BinaryOutDir is not part of the expected schema <class 'parflow.tools.database.generated.Solver'>
model.Solver.CLM.DailyRST = True
model.Solver.CLM.SingleFile = True
model.Solver.CLM.CLMDumpInterval = 1

Solver: Field BinaryOutDir is not part of the expected schema <class 'parflow.tools.database.generated.Solver'>


Next we specify the details of the meteorological forcing files that clm will read. 
<br />First we provide the name of the files and the directory they can be found in. 
<br />Next we specify that we are using 3D forcing files meaning that we have spatially distributed forcing with multiple time steps in every file. 
<br />Therefore we must also specify the number of times steps (MetFileNT) in every file, in this case 24. 
<br />Finally, we specify the initial value for the CLM counter.

In [69]:
model.Solver.CLM.MetFileName = "NLDAS"
model.Solver.CLM.MetFilePath = "../NLDAS"
model.Solver.CLM.MetForcing = "3D"
model.Solver.CLM.MetFileNT = 24
model.Solver.CLM.IstepStart = 1

This last set of CLM parameters refers to the physical properties of the system. Refer to § 6.1.36 for
details.

In [70]:
model.Solver.CLM.EvapBeta = "Linear"
model.Solver.CLM.VegWaterStress = "Saturation"
model.Solver.CLM.ResSat = 0.1
model.Solver.CLM.WiltingPoint = 0.12
model.Solver.CLM.FieldCapacity = 0.98
model.Solver.CLM.IrrigationType = "none"
model.Solver.CLM.RootZoneNZ = 4
model.Solver.CLM.SoiLayer = 4

#### Initial conditions: water pressure
Next we set the initial conditions for the domain. In this example we are using a pressure .pfb file that was obtained by spinning up the model in the workflow outlined in § 3.1.2. 
<br />Alternatively, the water table can be set to a constant value by changing the ICPressure.Type and setting a reference patch. 
<br />Again, the input file that is referenced here was was copied into the run directory at the top of this script.

In [71]:
model.ICPressure.Type = "PFBFile"
model.ICPressure.GeomNames = "domain"
model.Geom.domain.ICPressure.RefPatch = "top"
model.Geom.domain.ICPressure.FileName = "press.init.pfb"

# model.ICPressure.Type = "HydroStaticPatch"
# model.ICPressure.GeomNames = "domain"
# model.Geom.domain.ICPressure.Value = -15.0
# model.Geom.domain.ICPressure.RefGeom = "domain"
# model.Geom.domain.ICPressure.RefPatch = "top"

#### Outputs - Writing output (all pfb):
Now we specify what outputs we would like written. In this example we specify that we would like to write out CLM variables as well as Pressure and Saturation. 
<br />However, there are many options for this and you should change these options according to what type of analysis you will be performing on your results. 
<br />A complete list of print options is provided in § 6.1.32.

In [72]:
model.Solver.PrintSubsurfData = True
model.Solver.PrintPressure = True
model.Solver.PrintSaturation = False
model.Solver.PrintMask = True
model.Solver.PrintVelocities = False
model.Solver.PrintEvapTrans = False
model.Solver.CLM.SingleFile = False

model.Solver.WriteCLMBinary = False
model.Solver.PrintCLM = True
model.Solver.WriteSiloSpecificStorage = False
model.Solver.WriteSiloMannings = False
#model.Solver.WriteSiloMask = False
model.Solver.WriteSiloSlopes = False
model.Solver.WriteSiloSubsurfData = False
model.Solver.WriteSiloPressure = False
model.Solver.WriteSiloSaturation = False
model.Solver.WriteSiloEvapTrans = False
model.Solver.WriteSiloEvapTransSum = False
model.Solver.WriteSiloOverlandSum = False
#model.Solver.WriteSiloCLM = False

Exact solution specification for error calculations

In [73]:
model.KnownSolution = "NoKnownSolution"

#### Set solver parameters
Next we specify the solver settings for the ParFlow (§ 6.1.34). 
<br />First we turn on solver Richards and the terrain following grid.

In [74]:
# ParFlow Solution
model.Solver = "Richards"
model.Solver.TerrainFollowingGrid = True
#model.Solver.Nonlinear.VariableDz = False

We then set the max solver settings and linear and nonlinear convergence tolerance settings. 
<br />The linear system will be solved to a norm of 1e−8 and the nonlinear system will be solved to less than 1e−6. 
<br />Of note in latter key block is the EtaChoice of EtaConstant and that we use the analytical Jacobian (UseJacobian=True). 
<br />We are also using the PFMG preconditioner.

In [75]:
model.Solver.MaxIter = 25000
model.Solver.Drop = 1e-20
model.Solver.AbsTol = 1e-8
model.Solver.MaxConvergenceFailures = 8
model.Solver.Nonlinear.MaxIter = 1000
model.Solver.Nonlinear.ResidualTol = 1e-6

## new solver settings for Terrain Following Grid
model.Solver.Nonlinear.EtaChoice =  "EtaConstant"
model.Solver.Nonlinear.EtaValue = 0.001
model.Solver.Nonlinear.UseJacobian = True
model.Solver.Nonlinear.DerivativeEpsilon = 1e-16
model.Solver.Nonlinear.StepTol = 1e-15
model.Solver.Nonlinear.Globalization = "LineSearch"
model.Solver.Linear.KrylovDimension = 70
model.Solver.Linear.MaxRestarts = 2

model.Solver.Linear.Preconditioner = "PFMG"
#model.Solver.Linear.Preconditioner.PCMatrixType = "FullJacobian"

#### Distribute inputs
Next we distribute all the inputs over our domain geometry. 
<br />Note the slopes are 2D files, so we need to set our NZ to one and then back to 10 for the rest of the ParFlow inputs which are 3D.

In [76]:
model.ComputationalGrid.NZ =1
model.dist("slopex_LW.pfb")
model.dist("slopey_LW.pfb")
#model.dist("MANN_PFB") #if you have a Mannings pfb, it should be distributed over nz = 1

model.ComputationalGrid.NZ =10 #the rest of the inputs shoul dbe distributed over 3D space
model.dist("Indicator_LW_USGS_Bedrock.pfb")
model.dist("press.init.pfb")

#### Run Simulation

In [77]:
model.write()
model.run()


# ==============================================================================
# ParFlow directory
#  - /usr/local
# ParFlow version
#  - 3.10.0
# Working directory
#  - /data/LW/outputs
# ParFlow database
#  - LW_CLM_Ex4.pfidb
# ==============================================================================


# ==============================================================================
# ParFlow ran successfully 💦 💦 💦 
# ==============================================================================



#### Undistribute outputs